<a href="https://colab.research.google.com/github/mingchau/Generate_Chrome_Extension_i18n_Locale_Files/blob/main/Generate_Chrome_Extension_i18n_Locale_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# the key could be find in the Google doc spreadsheet url
spreadsheet = gc.open_by_key('1qSR49TD8heT9baOoEAPn-aOPWehHByBjh96jJWHean4')
worksheet = spreadsheet.get_worksheet(0)


# get_all_values gives a list of rows
# each element in the list is a row in the table
rows = worksheet.get_all_values()

# first row is the table head
# locale code starts from the 3rd column
locales = rows[0][2:]
print("Locales: ", locales)

# init the messages dictionary, to store all the messages
# the final messages dictionary should looks like this:
# {"en":{"messageKey":{"message":"xxx","description":"xxxx"}},"es":{...}}
messages = {}
 
# convert the messages to a pandas DataFrame and render
import pandas as pd
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
for locale in locales:
  messages[locale] = {}
  for row in df[['key', 'description', locale]].itertuples(index=False):
    messages[locale][row.key] = {'description': row.description, 'message': row[-1]}

Locales:  ['en', 'es']


In [8]:
# generate json files
import json
import os
for locale in locales:
  if not os.path.exists(f'./_locales'):
    os.mkdir(f'./_locales')
  if not os.path.exists(f'./_locales/{locale}'):
    os.mkdir(f'./_locales/{locale}')
  with open(f'./_locales/{locale}/messages.json', 'w', encoding='utf-8') as f:
    json.dump(messages[locale], f, ensure_ascii=False)

In [9]:
# zip the ./_locales directory for download
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile('locales.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('./_locales/', zipf)